In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os, optuna
import warnings

from tensorflow import keras
from google.colab import drive

drive.mount('/content/drive')
K = keras.backend
# Changing default dir

# optuna.logging.set_verbosity(optuna.logging.WARNING)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
!mkdir 'data'
!cp -r 'drive/MyDrive/maize-crop-diagnose' 'data'

In [4]:
# %cd '/content/drive/MyDrive/maize-crop-diagnose/data'
# !unzip train.zip

In [5]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        start_mom=0.95,
        min_mom=0.85,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.start_mom = start_mom
        self.min_mom = min_mom
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
            mom = self._interpolate(
                0, self.half_iteration, self.start_mom, self.min_mom
            )
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
            mom = self._interpolate(
                self.half_iteration,
                2 * self.half_iteration,
                self.min_mom,
                self.start_mom,
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
            mom = self.start_mom
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)
        K.set_value(self.model.optimizer.momentum, mom)

class OneCycleSchedulerNoMom(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [6]:
IMG_HEIGHT = 64 * 2
IMG_WIDTH = 48 * 2
BATCH_SIZE = 32
TRAIN_SIZE = 10000
TEST_SIZE = 3000
EPOCH = 50
DATA_DIR = "data/maize-crop-diagnose/data/train"

In [7]:
img_data = tf.keras.utils.image_dataset_from_directory(
        DATA_DIR,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
    )
num_classes = len(img_data.class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_set = img_data.take(TRAIN_SIZE).prefetch(buffer_size=AUTOTUNE)
test_set = img_data.skip(TRAIN_SIZE).take(TEST_SIZE)
val_set = img_data.skip(TRAIN_SIZE).skip(TEST_SIZE).prefetch(buffer_size=AUTOTUNE)

Found 14749 files belonging to 3 classes.


In [8]:
def objective(trial):
    K.clear_session()
    train_set, val_set = tf.keras.utils.image_dataset_from_directory(
        DATA_DIR,
        validation_split=0.2,
        subset="both",
        seed=42,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
    )
    num_classes = len(train_set.class_names)
    AUTOTUNE = tf.data.AUTOTUNE
    train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
    val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()
    # Hyperparameters for network architecture
    ## Number of filters
    filters_layer_1 = trial.suggest_categorical("filters_layer_1", [16, 32, 64])
    filters_layer_2 = trial.suggest_int("prop_filters_layer_2", 1, 4)
    filters_layer_3 = trial.suggest_int("prop_filters_layer_3", 1, 4)
    ## Kernel size
    kernel_layer_1 = trial.suggest_categorical("kernel_layer_1", [3, 5, 7])
    kernel_layer_2 = trial.suggest_categorical("kernel_layer_2", [3, 5, 7])
    kernel_layer_3 = trial.suggest_categorical("kernel_layer_3", [3, 5, 7])
    ## Activation function
    activation = trial.suggest_categorical("activation", ["elu", "selu"])
    if activation == "selu":
        kernel_initializer = "lecun_normal"
    else:
        kernel_initializer = "he_normal"
    ## Dropout rate
    dropout_rate = trial.suggest_float("dropout_rate", 0.25, 0.5)
    ## Dense layer size
    dense_size = trial.suggest_categorical("dense_size", [16, 32, 64, 128, 256, 512])
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Rescaling(1.0 / 255),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(
                filters_layer_1,
                kernel_layer_1,
                activation=activation,
                kernel_initializer=kernel_initializer,
            ),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.SpatialDropout2D(dropout_rate),
            tf.keras.layers.Conv2D(
                filters_layer_1 * filters_layer_2,
                kernel_layer_2,
                activation=activation,
                kernel_initializer=kernel_initializer,
            ),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.SpatialDropout2D(dropout_rate),
            tf.keras.layers.Conv2D(
                filters_layer_1 * filters_layer_2 * filters_layer_3,
                kernel_layer_3,
                activation=activation,
                kernel_initializer=kernel_initializer,
            ),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.SpatialDropout2D(dropout_rate),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(
                dense_size, activation=activation, kernel_initializer=kernel_initializer
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    # Hyperparameters for OneCycleScheduler
    max_lr = trial.suggest_float("max_lr", 1e-3, 1e-1)
    start_lr = trial.suggest_float("start_lr", max_lr * 0.01, max_lr * 0.8)
    last_lr = trial.suggest_float("last_lr", start_lr, max_lr)
    # Model definition
    onecycle = OneCycleSchedulerNoMom(
        TRAIN_SIZE // BATCH_SIZE * EPOCH,
        max_lr=max_lr,
        start_lr=start_lr,
        last_lr=last_lr
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        patience=5, restore_best_weights=True
    )
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )
    # Fitting model
    history = model.fit(
        train_set,
        validation_data=val_set,
        epochs=EPOCH,
        batch_size=BATCH_SIZE,
        callbacks=[early_stopping, onecycle],
    )
    # Evaluating and returning F1 score
    loss, acc = model.evaluate(val_set)
    return acc


study = optuna.create_study(
    storage="sqlite:///drive/MyDrive/maize-crop-diagnose/db_maize_models.sqlite3",
    study_name="cnn_onecycle_sgd",
    direction="maximize",
    load_if_exists=True,
)

study.optimize(objective, n_trials=100)
print(f"Best value: {study.best_value} (params: {study.best_params})")

[I 2023-06-07 23:03:54,298] Using an existing study with name 'cnn_onecycle_sgd' instead of creating a new one.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 99s 232ms/step - loss: 0.5733 - accuracy: 0.7686 - val_loss: 1.2351 - val_accuracy: 0.5354
Epoch 2/50
369/369 [==============================] - 17s 45ms/step - loss: 0.4495 - accuracy: 0.8208 - val_loss: 0.9147 - val_accuracy: 0.6460
Epoch 3/50
369/369 [==============================] - 17s 45ms/step - loss: 0.4171 - accuracy: 0.8340 - val_loss: 0.8125 - val_accuracy: 0.6812
Epoch 4/50
369/369 [==============================] - 17s 46ms/step - loss: 0.4008 - accuracy: 0.8429 - val_loss: 0.6963 - val_accuracy: 0.7416
Epoch 5/50
369/369 [==============================] - 17s 46ms/step - loss: 0.3731 - accuracy: 0.8536 - val_loss: 0.5230 - val_accuracy: 0.7952
Epoch 6/50
369/369 [==============================] - 17s 46ms/step - loss: 0.3639 - accuracy: 0.8593 - val_loss: 0.4764 - val_accuracy: 0.8149
Epoch 7/50
369/369 [========

[I 2023-06-07 23:15:56,373] Trial 14 finished with value: 0.9477788805961609 and parameters: {'activation': 'elu', 'dense_size': 128, 'dropout_rate': 0.4934076395227016, 'filters_layer_1': 32, 'kernel_layer_1': 5, 'kernel_layer_2': 5, 'kernel_layer_3': 7, 'last_lr': 0.029058761659346892, 'max_lr': 0.03674384541939109, 'prop_filters_layer_2': 4, 'prop_filters_layer_3': 3, 'start_lr': 0.027724501090552627}. Best is trial 10 with value: 0.9718548655509949.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 79s 202ms/step - loss: 0.4537 - accuracy: 0.8204 - val_loss: 0.7694 - val_accuracy: 0.6802
Epoch 2/50
369/369 [==============================] - 10s 27ms/step - loss: 0.3538 - accuracy: 0.8632 - val_loss: 0.2701 - val_accuracy: 0.8932
Epoch 3/50
369/369 [==============================] - 10s 27ms/step - loss: 0.3140 - accuracy: 0.8767 - val_loss: 0.2408 - val_accuracy: 0.9054
Epoch 4/50
369/369 [==============================] - 11s 29ms/step - loss: 0.2913 - accuracy: 0.8907 - val_loss: 0.2550 - val_accuracy: 0.9010
Epoch 5/50
369/369 [==============================] - 10s 28ms/step - loss: 0.2673 - accuracy: 0.9019 - val_loss: 0.1867 - val_accuracy: 0.9359
Epoch 6/50
369/369 [==============================] - 10s 27ms/step - loss: 0.2388 - accuracy: 0.9151 - val_loss: 0.2107 - val_accuracy: 0.9193
Epoch 7/50
369/369 [========

[I 2023-06-07 23:21:17,532] Trial 15 finished with value: 0.9721939563751221 and parameters: {'activation': 'elu', 'dense_size': 32, 'dropout_rate': 0.25718781252382916, 'filters_layer_1': 32, 'kernel_layer_1': 7, 'kernel_layer_2': 5, 'kernel_layer_3': 5, 'last_lr': 0.08326535676115024, 'max_lr': 0.09187652001152408, 'prop_filters_layer_2': 2, 'prop_filters_layer_3': 2, 'start_lr': 0.04965840885641326}. Best is trial 15 with value: 0.9721939563751221.


...

[I 2023-06-08 08:01:31,992] Trial 70 finished with value: 0.9776195287704468 and parameters: {'activation': 'elu', 'dense_size': 16, 'dropout_rate': 0.3149473262559441, 'filters_layer_1': 64, 'kernel_layer_1': 3, 'kernel_layer_2': 7, 'kernel_layer_3': 5, 'last_lr': 0.08260381066983141, 'max_lr': 0.08566759849468271, 'prop_filters_layer_2': 1, 'prop_filters_layer_3': 1, 'start_lr': 0.061729346076633704}. Best is trial 33 with value: 0.9810104966163635.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 85s 222ms/step - loss: 0.5424 - accuracy: 0.7849 - val_loss: 0.4419 - val_accuracy: 0.8386
Epoch 2/50
369/369 [==============================] - 18s 48ms/step - loss: 0.4314 - accuracy: 0.8361 - val_loss: 0.3280 - val_accuracy: 0.8762
Epoch 3/50
369/369 [==============================] - 17s 47ms/step - loss: 0.3878 - accuracy: 0.8564 - val_loss: 0.3047 - val_accuracy: 0.8786
Epoch 4/50
369/369 [==============================] - 17s 47ms/step - loss: 0.3586 - accuracy: 0.8692 - val_loss: 0.2452 - val_accuracy: 0.9125
Epoch 5/50
369/369 [==============================] - 17s 47ms/step - loss: 0.3532 - accuracy: 0.8674 - val_loss: 0.2872 - val_accuracy: 0.8861
Epoch 6/50
369/369 [==============================] - 18s 48ms/step - loss: 0.3366 - accuracy: 0.8777 - val_loss: 0.2169 - val_accuracy: 0.9234
Epoch 7/50
369/369 [========

[I 2023-06-08 08:14:41,460] Trial 71 finished with value: 0.9799932241439819 and parameters: {'activation': 'elu', 'dense_size': 16, 'dropout_rate': 0.3284468869364171, 'filters_layer_1': 64, 'kernel_layer_1': 3, 'kernel_layer_2': 7, 'kernel_layer_3': 5, 'last_lr': 0.06937721507983102, 'max_lr': 0.0737522422402427, 'prop_filters_layer_2': 2, 'prop_filters_layer_3': 1, 'start_lr': 0.056056360306309526}. Best is trial 33 with value: 0.9810104966163635.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 85s 222ms/step - loss: 0.5006 - accuracy: 0.7989 - val_loss: 0.4574 - val_accuracy: 0.8254
Epoch 2/50
369/369 [==============================] - 17s 47ms/step - loss: 0.4029 - accuracy: 0.8447 - val_loss: 0.3150 - val_accuracy: 0.8840
Epoch 3/50
369/369 [==============================] - 17s 47ms/step - loss: 0.3631 - accuracy: 0.8640 - val_loss: 0.2783 - val_accuracy: 0.8878
Epoch 4/50
369/369 [==============================] - 17s 47ms/step - loss: 0.3347 - accuracy: 0.8742 - val_loss: 0.2312 - val_accuracy: 0.9156
Epoch 5/50
369/369 [==============================] - 17s 47ms/step - loss: 0.3032 - accuracy: 0.8892 - val_loss: 0.2188 - val_accuracy: 0.9210
Epoch 6/50
369/369 [==============================] - 18s 48ms/step - loss: 0.2798 - accuracy: 0.8967 - val_loss: 0.1896 - val_accuracy: 0.9339
Epoch 7/50
369/369 [========

UnboundLocalError: ignored